In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass()

 ········


In [2]:
class LangGraphNode:
    rag_chain = None
    
    @classmethod
    def _load(cls):
        raise NotImplementedError()

    @classmethod
    def invoke(cls, *args, **kwargs):
        if cls.rag_chain is None:
            cls._load()
        return cls.rag_chain.invoke(*args, **kwargs)

# 1. Docs Retrieval

In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
class DocsRetrieval(LangGraphNode):
    URLS = [
        "https://lilianweng.github.io/posts/2023-06-23-agent/",
        "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
        "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
    ]
    
    @classmethod
    def _load(cls):
        docs = [WebBaseLoader(url).load() for url in cls.URLS]
        docs_list = [item for sublist in docs for item in sublist]
        
        text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
            chunk_size=250, chunk_overlap=0
        )
        doc_splits = text_splitter.split_documents(docs_list)
        
        # Add to vectorDB
        vectorstore = Chroma.from_documents(
            documents=doc_splits,
            collection_name="rag-chroma",
            embedding=OpenAIEmbeddings(),
        )
        cls.rag_chain = vectorstore.as_retriever()

# 2. Relevance Checker

In [5]:
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

In [6]:
class RelevanceGrader(LangGraphNode):
    # LLM
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    @classmethod
    def _load(cls):
        prompt = PromptTemplate(
            template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance
            of a retrieved document to a user question. If the document contains keywords related to the user question,
            grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
            Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
            Provide the binary score as a JSON with a single key 'score' and no premable or explanation.
             <|eot_id|><|start_header_id|>user<|end_header_id|>
            Here is the retrieved document: \n\n {document} \n\n
            Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
            """,
            input_variables=["question", "document"],
        )
        
        cls.rag_chain = prompt | cls.llm | JsonOutputParser()

# 3. Generator

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

In [8]:
class Generator(LangGraphNode):
    # LLM
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    @classmethod
    def _load(cls):
        # Prompt
        prompt = PromptTemplate(
            template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks.
            Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
            Use three sentences maximum and keep the answer concise <|eot_id|><|start_header_id|>user<|end_header_id|>
            Question: {question}
            Context: {context}
            Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
            input_variables=["question", "document"],
        )
        
        cls.rag_chain = prompt | cls.llm | StrOutputParser()

# 4. Hallucination Checker

In [9]:
class HallucinationGrader(LangGraphNode):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    @classmethod
    def _load(cls):
        # Prompt
        prompt = PromptTemplate(
            template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether
            an answer is grounded in / supported by a set of facts. Give a binary 'yes' or 'no' score to indicate
            whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a
            single key 'score' and no preamble or explanation. <|eot_id|><|start_header_id|>user<|end_header_id|>
            Here are the facts:
            \n ------- \n
            {documents}
            \n ------- \n
            Here is the answer: {generation}  <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
            input_variables=["generation", "documents"],
        )
        
        cls.rag_chain = prompt | cls.llm | JsonOutputParser()

# 5. SearchEngine

In [10]:
from tavily import TavilyClient
from langchain_core.documents import Document

In [11]:
tavily_api_key = getpass.getpass()

 ········


In [12]:
class SearchEngine(LangGraphNode):
    tavily = TavilyClient(api_key=tavily_api_key)

    @classmethod
    def _load(cls):
        pass
        
    @classmethod
    def invoke(cls, *args, **kwargs):
        response = cls.tavily.search(*args, **kwargs)
        context = [{"url": obj["url"], "title": obj["title"], "content": obj["content"]} for obj in response['results']]
        docs = [Document(page_content=c["content"], metadata={"url": c["url"], "title": c["title"]}) for c in context]

        return docs

# 6. Final Lang Graph

In [13]:
class LangGraph:
    generation_cnt = 10
    search_cnt = 10
    
    @classmethod
    def _invoke(cls, question, docs):
        success = False
        answer = title = url = ""
        for doc in docs:
            doc_txt = doc.page_content
            relevance_grade = RelevanceGrader.invoke({"question": question, "document": doc_txt})
            if relevance_grade["score"] != "yes":
                continue
                
            success = True
            title = doc.metadata["title"]
            if "url" in doc.metadata:
                url = doc.metadata["url"]
            elif "source" in doc.metadata:
                url = doc.metadata["source"]

            hallucinated = True
            cnt = 0
            while cnt < cls.generation_cnt:
                cnt += 1
                answer = Generator.invoke({"question": question, "context": doc_txt})
                answer_grade = HallucinationGrader.invoke({"generation": answer, "documents": doc})
                if answer_grade["score"] == "yes":
                    hallucinated = False
                    break
                    
            if hallucinated:
                answer = f"(Hallucinated Answer){answer}"

        return success, answer, title, url

    @classmethod
    def invoke(cls, question):
        res = ""
        cnt = 0
        while cnt < cls.search_cnt:
            cnt += 1
            docs = DocsRetrieval.invoke(question)
            success, answer, title, url = cls._invoke(question, docs)
            if not success:
                docs = SearchEngine.invoke(question)
                success, answer, title, url = cls._invoke(question, docs)
            res = f"*Answer*: {answer}\n\n*Url*: {url}\n\n*Title*: {title}"
            if success:
                break
        return res

In [14]:
print(LangGraph.invoke("agent memory"))

*Answer*: Agent memory can be categorized into short-term and long-term memory. Short-term memory involves in-context learning, while long-term memory allows the agent to retain and recall information over extended periods, often using an external vector store. Additionally, agents can utilize external APIs to access information beyond their pre-trained knowledge.

*Url*: https://lilianweng.github.io/posts/2023-06-23-agent/

*Title*: LLM Powered Autonomous Agents | Lil'Log


In [15]:
print(LangGraph.invoke("'꽁꽁 얼어붙은 한강 위로 고양이가 걸어다닙니다' 밈에 대해서 알려줘"))

*Answer*: '꽁꽁 얼어붙은 한강 위로 고양이가 걸어다닙니다'는 2024년에 화제가 된 인터넷 밈으로, 2021년 12월 27일 MBN 뉴스7의 보도 장면에서 유래했습니다. 이 장면은 얼어붙은 한강 위를 걷는 고양이를 담고 있으며, 이후 다양한 패러디와 변형이 만들어졌습니다. 이 밈은 주로 고양이의 귀여움과 상황의 아이러니를 강조하는 데 사용됩니다.

*Url*: https://www.etoday.co.kr/news/view/2352921

*Title*: 꽁꽁 얼어붙은 한강 위로 고양이가 걸어다닙니다…뉴스밈 또 터졌다 [요즘, 이거] - 이투데이
